In [1]:
import sys
sys.path.append(r"C:\Users\Felix\Documents\.Uni\Project\Git\tum-ase-project-2025-1_2\optimization")

import numpy as np
import pandas as pd
import plotly.express as px
from trained_surrogate import predict_rf_and_weight

# --- Your base input vector (25-dimensional) ---
base_vector = [
    # web_height_1-5
    28.44463121633852,
    29.14981637771977,
    28.02501643353222,
    27.73787754413,
    28.91337248860228,
    # flange_width_1-5
    27.93095091281996,
    24.98807164598568,
    22.72418017778071,
    26.47761847442586,
    24.08167965295736,
    # lip_height_1-5
    6.149014513434929,
    6.49403893013365,
    6.053262930575113,
    5.403164790931185,
    6.169567376819412,
    # thickness_1-5
    4.39696521677382,
    4.80698157651496,
    5.292843125043317,
    5.299202465485834,
    4.481495697075892,
    # skin_thickness_1-5
    4.788768125034848,
    4.950913879455937,
    4.98509771349832,
    4.89899727170224,
    5.699931076340307
]

# --- Define percent deviation (+/- as a fraction, e.g., 0.06 for 6%) ---
deviation = 0.005 

# --- Number of samples to generate ---
n_samples = 2


# --- Calculate per-variable bounds ---
base = np.array(base_vector)
lower_bounds = base * (1 - deviation)
upper_bounds = base * (1 + deviation)

# Optional: enforce min/max if you wish to not go out of physical/realistic range
#abs_min = np.array([20]*5 + [20]*5 + [4.5]*5 + [4.2]*5 + [4.6]*5)
#abs_max = np.array([30]*5 + [30]*5 + [6.5]*5 + [5.6]*5 + [5.7]*5)
#lower_bounds = np.maximum(lower_bounds, abs_min)
#upper_bounds = np.minimum(upper_bounds, abs_max)

# --- Generate random samples in this local box ---
random_vectors = np.random.uniform(lower_bounds, upper_bounds, size=(n_samples, 25))

results = []
for x in random_vectors:
    rf_est, weight_est = predict_rf_and_weight(x)
    if 1.00 <= rf_est <= 1.05 and weight_est < 28.5:
        results.append(list(x) + [rf_est, weight_est])

# --- Save results to CSV ---
columns = [f'input_{i+1}' for i in range(25)] + ['predicted_min_nonstrength_rf', 'predicted_weight']
df_results = pd.DataFrame(results, columns=columns)
df_results.to_csv('surrogate_local_candidates.csv', index=False)
print(f"Saved {len(df_results)} samples in local region to 'surrogate_local_candidates.csv'.")

# --- Plot the results ---
fig = px.scatter(
    df_results, x='predicted_weight', y='predicted_min_nonstrength_rf',
    labels={'predicted_weight': 'Predicted Weight (kg)', 'predicted_min_nonstrength_rf': 'Predicted min_nonstrength_rf'},
    title=f"Surrogate Model: Local Scan (+/-{int(deviation*100)}%) Region (RF 1.00–1.05, Weight < 28.5 kg)"
)
fig.show()


Saved 2 samples in local region to 'surrogate_local_candidates.csv'.


In [2]:
import sys
sys.path.append(r"C:\Users\Felix\Documents\.Uni\Project\Git\tum-ase-project-2025-1_2\optimization")

import numpy as np
import pandas as pd
import plotly.express as px
from trained_surrogate import predict_rf_and_weight

# --- The PCA-box-derived bounds from the analysis ---
lower_bounds = np.array([
    25.000000, 25.000000, 25.000000, 25.000000, 25.000000,          # web_height_1-5
    20.704072, 20.716048, 20.068515, 21.771497, 20.049914,           # flange_width_1-5
    2.000000, 2.000000, 2.000000, 2.000000, 2.000000,                # lip_height_1-5
    2.610333, 2.610333, 2.610333, 2.141920, 2.457918,                # thickness_1-5
    4.579384, 4.735956, 4.755781, 4.712407, 5.431102                 # skin_thickness_1-5
])
upper_bounds = np.array([
    36.002673, 32.414907, 37.009215, 37.398856, 35.282474,           # web_height_1-5
    31.442635, 29.907754, 29.907754, 33.965549, 31.763556,           # flange_width_1-5
    9.958820, 11.300201, 12.134010, 9.060187, 11.751471,             # lip_height_1-5
    5.498370, 5.462907, 5.478143, 5.587557, 5.466221,                # thickness_1-5
    15.000000, 15.000000, 15.000000, 15.000000, 14.970353            # skin_thickness_1-5
])

n_samples = 200000

# --- Generate random samples in the PCA-box bounds ---
random_vectors = np.random.uniform(lower_bounds, upper_bounds, size=(n_samples, 25))

results = []
for x in random_vectors:
    rf_est, weight_est = predict_rf_and_weight(x)
    if 1.00 <= rf_est <= 1.09 and weight_est < 28.5:
        results.append(list(x) + [rf_est, weight_est])

# --- Save results to CSV ---
columns = [f'input_{i+1}' for i in range(25)] + ['predicted_min_nonstrength_rf', 'predicted_weight']
df_results = pd.DataFrame(results, columns=columns)
df_results.to_csv('surrogate_pca_box_candidates.csv', index=False)
print(f"Saved {len(df_results)} samples from PCA box to 'surrogate_pca_box_candidates.csv'.")

# --- Plot the results ---
fig = px.scatter(
    df_results, x='predicted_weight', y='predicted_min_nonstrength_rf',
    labels={'predicted_weight': 'Predicted Weight (kg)', 'predicted_min_nonstrength_rf': 'Predicted min_nonstrength_rf'},
    title=f"Surrogate Model: PCA Box Scan (RF 1.00–1.05, Weight < 28.5 kg)"
)
fig.show()


Saved 18993 samples from PCA box to 'surrogate_pca_box_candidates.csv'.
